### Add colored sparks on video

28.02.2022: questo codice viene da un vecchio file che avevo creato per aggiungere delle preds colorate sui video originali

TODO: adattarlo al codice corrente

Add colored annotations and predictions (with transparency) on top of sample video

Created the 29th November 2019

For a given sample video needs:
- sample video
- mask with sparks annotations
- mask with network predicitions for sparks

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import math
import pandas as pd
import imageio
import itertools
from PIL import Image
from scipy import spatial, optimize

from metrics_tools import nonmaxima_suppression, Metrics, correspondences_precision_recall, reduce_metrics
#from save_predictions_as_videos import find_file

In [25]:
def paste_annotations_on_video(video, colored_mask):
    # video is a RGB video, list of PIL images
    # colored_mask is a RGBA video, list of PIL images
    for frame,ann in zip(video, colored_mask):
        frame.paste(ann, mask = ann.split()[3])

def add_colored_annotations_to_video(annotations,video,color,transparency=50,radius=4):
    # annotations is a list of t,x,y coordinates
    # video is a RGB video, list of PIL images
    # color is a list of 3 RGB elements
    mask_shape = (len(video), video[0].size[1], video[0].size[0], 4)
    colored_mask = np.zeros(mask_shape, dtype=np.uint8)
    for pt in annotations:
        colored_mask = color_ball(colored_mask,pt,radius,color,transparency)
    colored_mask = [Image.fromarray(frame).convert('RGBA') for frame in colored_mask]

    paste_annotations_on_video(video, colored_mask)
    return video

def l2_dist(p1,p2):
    # p1 = (t1,y1,x1)
    # p2 = (t2,y2,x2)
    t1,y1,x1 = p1
    t2,y2,x2 = p2
    return math.sqrt(math.pow((t1-t2),2)+math.pow((y1-y2),2)+math.pow((x1-x2),2))

def ball(c,r):
    # r scalar
    # c = (t,y,x)
    # returns coordinates c' around c st dist(c,c') <= r
    t,y,x = c
    t_vect = np.linspace(t-r,t+r, 2*r+1, dtype = int)
    y_vect = np.linspace(y-r,y+r, 2*r+1, dtype = int)
    x_vect = np.linspace(x-r,x+r, 2*r+1, dtype = int)

    cube_idxs = list(itertools.product(t_vect,y_vect,x_vect))
    ball_idxs = [pt for pt in cube_idxs if l2_dist(c, pt) <= r]

    return ball_idxs

def color_ball(mask,c,r,color,transparency=50):
    color_idx = ball(c,r)
    # mask boundaries
    duration, height, width, _ = np.shape(mask)

    for t,y,x in color_idx:
        if 0 <= t and t < duration and 0 <= y and y < height and 0 <= x and x < width:
            mask[t,y,x] = [*color, transparency]

    return mask

### Load movies, annotations & training predictions (only sparks needed)

In [8]:
# physiological params
PIXEL_SIZE = 0.2 # 1 pixel = 0.2 um x 0.2 um
global MIN_DIST_XY
MIN_DIST_XY = round(1.8 / PIXEL_SIZE) # min distance in space between sparks
TIME_FRAME = 6.8 # 1 frame = 6.8 ms
global MIN_DIST_T
MIN_DIST_T = round(20 / TIME_FRAME) # min distance in time between sparks

In [9]:
# parameters
ignore_frames = 6

t_detection_sparks = 0.7
min_radius_sparks = 2

transparency = 45

In [10]:
from dataset_tools import load_movies_ids, load_predictions

In [19]:
# sample training name
training_name = "focal_loss_gamma_5_ubelix"
epoch = 100000
metrics_folder = os.path.join("trainings_validation",training_name,"colored_sparks")

os.makedirs(metrics_folder, exist_ok=True)

In [12]:
# load annotations and sparks
ys, sparks, puffs, _ = load_predictions(training_name, epoch, metrics_folder)

In [13]:
# get movies names as tuple
movies_ids = tuple(ys.keys())
print(movies_ids)

('05', '10', '15', '20', '25', '32', '34', '40', '45')


In [14]:
# load movies
data_folder = os.path.join("..","data","raw_data_and_processing","original_movies")
xs = load_movies_ids(data_folder, movies_ids)

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


In [15]:
from metrics_tools import get_sparks_locations_from_mask, process_spark_prediction, correspondences_precision_recall

In [16]:
# use this to consider sum of puffs and sparks preds inside of puffs
sum_sparks_puffs = True

In [17]:
# if white_background, add colored sparks to white background
# if not white_bacjground, add colored sparks to original movies
white_background = True

In [24]:
for movie_id in movies_ids:
    print("Processing video", movie_id, "...")
    
    # normalize sample movie
    sample_video = xs[movie_id]
    
    if white_background:
        sample_video.fill(1) # the movie will be white
    sample_video = 255*(sample_video/sample_video.max())
    
    # get annotated sparks locations
    coords_true = get_sparks_locations_from_mask(ys[movie_id],
                                                 min_dist_xy=MIN_DIST_XY,
                                                 min_dist_t=MIN_DIST_T,)
    
    # get predicted sparks locations
    spark_preds = sparks[movie_id]
    if sum_sparks_puffs:
        # set puff boundarier
        t_puffs_lower = 0.3
        t_puffs_upper = 0.65 # = t detection puffs
        puff_preds = puffs[movie_id]
        # compute region where 0.3 <= puffs <= 0.65
        binary_puffs_sparks = np.logical_and(puff_preds <= t_puffs_upper,
                                             puff_preds >= t_puffs_lower)
        # sum value of sparks and puffs in this region
        spark_preds = spark_preds + binary_puffs_sparks * puff_preds

    coords_preds = process_spark_prediction(pred=spark_preds, 
                                            t_detection=t_detection_sparks,
                                            min_dist_xy=MIN_DIST_XY,
                                            min_dist_t=MIN_DIST_T,
                                            min_radius=min_radius_sparks,
                                           )
    
    # Compute correspondences between annotations and predictions
    ''' OLD
    distances = spatial.distance_matrix(coords_true, coords_preds)
    distances[distances > match_distance] = 9999999
    row_ind, col_ind = optimize.linear_sum_assignment(distances)

    paired_true = [coords_true[i].tolist() for i,j in zip(row_ind,col_ind) if distances[i,j]<=match_distance]
    paired_preds = [coords_preds[j].tolist() for i,j in zip(row_ind,col_ind) if distances[i,j]<=match_distance]
    '''
    
    paired_true, paired_preds, false_positives, false_negatives = correspondences_precision_recall(coords_true,
                                                                                                   coords_preds,
                                                                                                   MIN_DIST_T,
                                                                                                   MIN_DIST_XY,
                                                                                                   return_pairs_coords=True
                                                                                                  )

    # Add colored annotations to video

    rgb_video = [Image.fromarray(frame).convert('RGB') for frame in sample_video]

    annotated_video = add_colored_annotations_to_video(paired_true, rgb_video, [0,255,0], 0.8*transparency)
    annotated_video = add_colored_annotations_to_video(paired_preds, annotated_video, [0,255,200], 0.8*transparency)
    annotated_video = add_colored_annotations_to_video(false_positives, annotated_video, [255,255,0], transparency)
    annotated_video = add_colored_annotations_to_video(false_negatives, annotated_video, [255,0,0], transparency)

    annotated_video = [np.array(frame) for frame in annotated_video]
    
    # set saved movies filenames
    white_background_fn = "_white_backgroud" if white_background else ""
    sum_sparks_fn = "_sum_puffs" if sum_sparks_puffs else ""
    
    imageio.volwrite(os.path.join(metrics_folder,
                                  training_name+"_"+str(epoch)+"_"+movie_id+"_colored_sparks"+sum_sparks_fn+white_background_fn+".tif"),
                     annotated_video)

Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video 40 ...
Processing video 45 ...


### Write all script parameters to file

In [31]:
import datetime

In [32]:
file_path = os.path.join(metrics_folder,"parameters.txt")

with open(file_path, 'w') as f:
    f.write(f"{datetime.datetime.now()}\n\n")
    
    f.write("Phyisiological parameters\n")
    f.write(f"Pixel size: {PIXEL_SIZE} um\n")
    f.write(f"Min distance (x,y): {MIN_DIST_XY} pixels\n")
    f.write(f"Time frame: {TIME_FRAME} ms\n")
    f.write(f"Min distance t: {MIN_DIST_T} pixels\n\n")
    
    f.write("Training parameters\n")
    f.write(f"Training name: {training_name}\n")
    f.write(f"Loaded epoch: {epoch}\n")
    f.write(f"Dataset folder: {data_folder}\n")
    f.write(f"Movies analysed for coloured sparks: {movies_ids}\n")
    f.write(f"Num frames ignored by loss function: {ignore_frames}\n\n")
    
    f.write("Sparks detection parameters\n")
    f.write(f"Min threshold for sparks detection: {t_detection_sparks}\n")
    f.write(f"Min radius of valid spark predictions: {min_radius_sparks}\n")
    f.write(f"Using puffs values for sparks detection: {sum_sparks_puffs}\n")
    if sum_sparks_puffs:
        f.write(f"Min puffs' threshold for sparks over puffs detection: {t_puffs_lower}\n")
        f.write(f"Max puffs' threshold for sparks over puffs detection: {t_puffs_upper}\n\n")
        
    f.write("Coloured sparks parameters\n")
    f.write(f"Saved coloured sparks path: {metrics_folder}\n")
    f.write(f"Coloured sparks' transparency: {transparency}\n")
    f.write(f"Using white background instead of original movies: {white_background}\n")    